In [ ]:
#### Data processing

In [ ]:
import numpy as np
import pickle
from scipy import signal
from finn.basic import downsampling, common_average_rereferncing
from finn.cleansing import bad_channel_identification
import pickle
from pylab import *
from scipy.integrate import simps


In [ ]:
def step_0(x, fs):
    b, a = signal.butter(N=4, Wn=300, btype='lowpass', fs=fs)
    z = signal.lfilter(b, a, x)
    return z

def bandpass(x, freq):
    b, a = signal.butter(N=4, Wn=[0.53,50], btype='bandpass', fs=freq)
    z = signal.lfilter(b, a, x)
    return z

def notch(x, freq):
        b, a = signal.iirnotch(50, 50/5, fs=freq)
        z = signal.lfilter(b, a, x)
        return z

def epoch(data, t1, t2, fs, t_int, num_ch):
    d1 = t1 * fs
    d2 = t2 * fs
    data = data[:,d1:d2]
    d3 = t_int * fs
    num_epoch = int((t2 - t1)/t_int) 
    epoch_data = [[0]] * len(data)
    for i in range(len(data)):
        epoch_data[i] = [data[i,0: d3]]
        for j in range(d3, len(data[i]), d3):
            epoch_data[i].append(data[i,j:j + d3])
    epoch_data = np.array(epoch_data)
    epoch_data = np.transpose(epoch_data, (1,0,2))
    return epoch_data

def epoch_extract_beta(data, fs):
    epoch_power = np.zeros([len(data),len(data[0])])
    for i in range(len(data)):
        for j in range(len(data[i])):
            power = signal.welch(data[i][j], fs, nfft = fs, nperseg = fs)[1]
            epoch_power[i,j] = np.sum(power[13:32])/np.sum(power[0:300])
    return epoch_power

In [ ]:
def single_file(path, path_info, fs, freq):
    with open(path, 'rb') as file:
        data = np.load(file)
        data = data['arr_0']

    run_00_hdr = open(path_info, 'rb')
    model_input = pickle.load(run_00_hdr)
    ch_names = model_input['chNames']

    ## Step 0: Lowpass filter before downsample
    step0 = np.apply_along_axis(step_0, 1, data, fs)

    ## Step 1: Downsample
    func = lambda x: downsampling.run(x, src_freq=fs, tgt_freq=freq)
    step1 = np.apply_along_axis(func, 1, step0)
    
    
    ## Step 1b: Remove Bad Channels 
    try:
        x = bad_channel_identification.run(
        step1,
        ch_names=ch_names,
        fs=[freq]*len(step1),
        ref_areas=[[60,100]],
        broadness=3,
        visual_inspection=False
        )
        ch_names = [ch_names[i] for i in x[0]]
        step1b = [step1[i] for i in x[0]]
        if step1b != []:
            step1 = step1b
    except:
        print("Bad channel identification failed")
    
    num_ch = len(ch_names)
    

    ## Step 2: Bandpass filter (0.53 - 50 Hz)

    step2 = np.apply_along_axis(bandpass, 1, step1, freq)

    ## Step 3: Notch filter at 50 Hz

    step3 = np.apply_along_axis(notch, 1, step2, freq)

    ## Step 4: common avg re-ref

    step4 = common_average_rereferncing.run(step3)

    ## Step 5: Epoch and Split Data

    step5a = epoch(step4, 4, 26, freq, 2, num_ch)
    step5b = epoch(step4, 54, 76,freq, 2, num_ch)

    # Step 6: Extract power in the beta-frequency band and normalize by power between 0 and 300Hz

    step6a = epoch_extract_beta(step5a, freq)
    step6b = epoch_extract_beta(step5b, freq)

    return step6a, step6b

In [ ]:
import os
from tqdm import tqdm

fs = 5000
freq = 600

rootdir = r'C:\Users\laurawheeler\Desktop\beta'
destdir = r'C:\Users\laurawheeler\Desktop\beta_processed'

path_run = 'hi'
path_info_run = 'hello'

for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        filepath = subdir + os.sep + file

        if filepath.endswith(".npz"):
            path = filepath
            path_run = path[path.index('run'):]
            path_run = path_run[:6]

        if filepath.endswith(".pkl"):
            path_info = filepath
            path_info_run = path_info[path_info.index('run'):]
            path_info_run = path_run[:6]
        
        if path_run == path_info_run:
            try:
                data1, data2 = single_file(path, path_info, fs, freq)
                path = path[path.index('beta')+4:-8]
                path_a = destdir + path + "_a.csv"
                np.savetxt(path_a, data1, delimiter=",")
                path_b = destdir + path + "_b.csv"
                np.savetxt(path_b, data2, delimiter=",")
            except:
                print(path, "Encountered an error")

In [ ]:
#### Get channel names in order from each trial

In [ ]:
import numpy as np
import pickle
from scipy import signal
from finn.basic import downsampling, common_average_rereferncing
from finn.cleansing import bad_channel_identification
import pickle
from pylab import *
from scipy.integrate import simps

In [ ]:
def step_0(x, fs):
    b, a = signal.butter(N=4, Wn=300, btype='lowpass', fs=fs)
    z = signal.lfilter(b, a, x)
    return z

def bandpass(x, freq):
    b, a = signal.butter(N=4, Wn=[0.53,50], btype='bandpass', fs=freq)
    z = signal.lfilter(b, a, x)
    return z

def notch(x, freq):
        b, a = signal.iirnotch(50, 50/5, fs=freq)
        z = signal.lfilter(b, a, x)
        return z

def epoch(data, t1, t2, fs, t_int):
    d1 = t1 * fs
    d2 = t2 * fs
    data = data[:,d1:d2]
    d3 = t_int * fs
    num_epoch = int((t2 - t1)/t_int) 
    epoch_data = [[0]] * len(data)
    for i in range(len(data)):
        epoch_data[i] = [data[i,0: d3]]
        for j in range(d3, len(data[i]), d3):
            epoch_data[i].append(data[i,j:j + d3])
    epoch_data = np.array(epoch_data)
    epoch_data = np.transpose(epoch_data, (1,0,2))
    return epoch_data

def epoch_extract_beta(data, fs):
    epoch_power = np.zeros([len(data),len(data[0])])
    for i in range(len(data)):
        for j in range(len(data[i])):
            power = signal.welch(data[i][j], fs, nfft = fs, nperseg = fs)[1]
            epoch_power[i,j] = np.sum(power[13:32])/np.sum(power[0:300])
    return epoch_power

In [ ]:
def single_file(path, path_info, fs, freq):
    with open(path, 'rb') as file:
        data = np.load(file)
        data = data['arr_0']

    run_00_hdr = open(path_info, 'rb')
    model_input = pickle.load(run_00_hdr)
    ch_names = model_input['chNames']

    ## Step 0: Lowpass filter before downsample
    step0 = np.apply_along_axis(step_0, 1, data, fs)

    ## Step 1: Downsample
    func = lambda x: downsampling.run(x, src_freq=fs, tgt_freq=freq)
    step1 = np.apply_along_axis(func, 1, step0)
    
    
    ## Step 1b: Remove Bad Channels 
    try:
        x = bad_channel_identification.run(
        step1,
        ch_names=ch_names,
        fs=[freq]*len(step1),
        ref_areas=[[60,100]],
        broadness=3,
        visual_inspection=False
        )
        ch_names = [ch_names[i] for i in x[0]]
        step1b = [step1[i] for i in x[0]]
        if step1b != []:
            step1 = step1b
        else:
            ch_names = model_input['chNames']
    except:
        print("Bad channel identification failed")

    return ch_names

In [ ]:
import os
from tqdm import tqdm

fs = 5000
freq = 600

rootdir = r'C:\Users\laurawheeler\Desktop\beta'
destdir = r'C:\Users\laurawheeler\Desktop\beta_processed'

path_run = 'hi'
path_info_run = 'hello'

file_channels = []

for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        filepath = subdir + os.sep + file

        if filepath.endswith(".npz"):
            path = filepath
            path_run = path[path.index('run'):]
            path_run = path_run[:6]

        if filepath.endswith(".pkl"):
            path_info = filepath
            path_info_run = path_info[path_info.index('run'):]
            path_info_run = path_run[:6]
        
        if path_run == path_info_run:
            try:
                ch_names = single_file(path, path_info, fs, freq)
                path = path[path.index('beta')+4:-8]
                path_a = destdir + path + "_a.csv"
                file_channels.append([path_a[41:],ch_names])
                path_b = destdir + path + "_b.csv"
                file_channels.append([path_b[41:],ch_names])
            except:
                print(path, "Encountered an error")

with open(r'C:\Users\laurawheeler\Desktop\ch_names.ob', 'wb') as fp:
    pickle.dump(file_channels,fp)

In [ ]:
##ANOVA Analysis

In [1]:
import statsmodels.api as sm
import numpy as np
import pickle
import pandas as pd
from statsmodels.formula.api import ols
import os
BASE_PATH = r'C:\Users\laurawheeler\Downloads\ANOVA\ANOVA'

In [ ]:

chNames = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 
            'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 
            'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 
            'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT9', 'FT7', 'FC3', 'FC4', 'FT8', 'FT10', 
            'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 
            'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'LT']

all_p_vals = []

for col_name in chNames:

    df_all = pd.DataFrame([])
    for i in os.listdir(f"{BASE_PATH}/beta_processed"):  # for each participant
        if not i.isdigit():
            continue
        
        path = f"{BASE_PATH}/beta_processed/{i}/raw/D1/left_r/"
        for file in os.listdir(path):  # for each participant "run"

            metadata_path = f"{BASE_PATH}/pickles/{i}/raw/D1/left_r/{file[:-10]}hdr.pkl"
            metadata = pickle.load(open(metadata_path, 'rb'))

            # get the index of the channel name we're interested in
            index = metadata['chNames'].index(col_name)
            if index == -1:
                print(f"skipping {file} due to missing column")
                continue
            
            # load the data for the channel we want
            data = np.loadtxt(open(path + file, "rb"), delimiter=",")[:,index]
            
            # build the dataframe for this run
            df = pd.DataFrame(data, columns=[col_name])
            df['power'] = df[col_name]
            df['stim'] = metadata['stimInt']
            df['clin_score'] = metadata['clinScore']
            df['patient_id'] = metadata['patientId']
            df['move'] = 0 if 'a.csv' in file else 1
            df['samp_id'] = np.arange(1, len(df) + 1)
            df['file_id'] = 1
            df_all = pd.DataFrame.append(df_all, df)

    # Apply the 3 way ANOVA
    model = ols(
        'power ~ C(move) + C(stim) + C(clin_score) + C(move):C(stim) + C(clin_score):C(stim)'\
        '+ C(move):C(clin_score) + (1|patient_id) + (1|file_id) + (1|samp_id)',
        data=df_all
    ).fit()

    table = sm.stats.anova_lm(model, typ=3)
    table = table[1:7]
    p_vals = table['PR(>F)']
    p_vals = p_vals.to_list()
    all_p_vals.append(p_vals)
 

with open(r'C:\Users\laurawheeler\Desktop\pppppp.ob', 'wb') as fp:
    pickle.dump(all_p_vals,fp)

In [ ]:
### Creating the Dataframe for JMP

In [ ]:
# Load all participant data for all electrodes into one dataframe to use in JMP

chNames = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 
            'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 
            'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 
            'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT9', 'FT7', 'FC3', 'FC4', 'FT8', 'FT10', 
            'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 
            'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'LT']

df_all_all = pd.DataFrame([])
for col_name in tqdm(chNames):
    df_all = pd.DataFrame([])
    for i in os.listdir(f"{BASE_PATH}/beta_processed"):  # for each participant
        if not i.isdigit():
            continue

        path = f"{BASE_PATH}/beta_processed/{i}/raw/D1/left_r/"
        for file in os.listdir(path):  # for each participant "run"

            metadata_path = f"{BASE_PATH}/pickles/{i}/raw/D1/left_r/{file[:-10]}hdr.pkl"
            metadata = pickle.load(open(metadata_path, 'rb'))

            # get the index of the channel name we're interested in
            index = metadata['chNames'].index(col_name)
            if index == -1:
                print(f"skipping {file} due to missing column")
                continue

            # load the data for the channel we want
            data = np.loadtxt(open(path + file, "rb"), delimiter=",")[:,index]

            # build the dataframe for this run
            df = pd.DataFrame(data, columns=['power'])
            df['stim'] = metadata['stimInt']
            df['clin_score'] = metadata['clinScore']
            df['patient_id'] = metadata['patientId']
            df['move'] = 0 if 'a.csv' in file else 1
            df['samp_id'] = np.arange(1, len(df) + 1)
            df['file_id'] = 1
            df_all = pd.DataFrame.append(df_all, df)

    df_all_all = pd.DataFrame.append(df_all_all, df_all)
df_all_all.to_csv("output_updated.csv")

In [ ]:
### Average beta power in each channel

In [ ]:
df_all['power'].mean() 

In [ ]:
### Topoplots

In [17]:
import statsmodels.api as sm
import random
import pickle
import finn.visualization.topoplot as tp
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Qt5agg")
import matplotlib.pyplot as plt
from statsmodels.stats import multitest


with open (r'C:\Users\laurawheeler\Desktop\pppppp.ob', 'rb') as fp:
    p_vals = pickle.load(fp)

chNames = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 
            'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 
            'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'AF7', 'AF3', 
            'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FT9', 'FT7', 'FC3', 'FC4', 'FT8', 'FT10', 
            'C5', 'C1', 'C2', 'C6', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P5', 'P1', 'P2', 'P6', 
            'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'LT']


#df = pd.DataFrame(p_vals)

headings = ['move','stim', 'clin_score', 'move:stim', 'clin_score:stim', 'move:clin_score']

df = pd.DataFrame(p_vals, index = chNames, columns = headings)

#df.to_csv(r'C:\Users\laurawheeler\Desktop\df.csv', encoding='utf-8', index=True)
    
p_vals = df.sort_index().values.tolist()

#pd.set_option("display.max_rows", None, "display.max_columns", None)

#df.sort_index()


In [ ]:
sig_p_vals = []

for j in range(len(p_vals)):
    sig = []
    for v in p_vals[j]:
        if v < 0.01:
            sig.append(1)
        else:
            sig.append(0)
    sig_p_vals.append(sig)

In [ ]:
corr_sig = []

for i in range(6):
    reject, pvals_corrected, _, _ = multitest.multipletests(np.array(p_vals)[:,i], alpha=0.01, method='hs')
    corr_sig.append(reject)

In [ ]:
path = r'C:\Users\laurawheeler\Desktop\channel_power.csv'
df = pd.read_csv(path)
chNames = df['Channel'].tolist()
pow = df['Power'].tolist()

In [ ]:
topo = tp.topoplot("EEG")

In [ ]:
for j in range(6):
    sig_lst = []
    for p_ch in sig_p_vals:
        sig_lst.append(p_ch[j]) 
    #print(j)
    topo.run(np.asarray([pow, sig_lst, corr_sig[j]]).transpose(), chNames, omit_channels=['RT'], substitute_channels=[{"src":["Fp1"], "tgt" : "Fp1"}], v_min=0, v_max=0.2, v_border_values=[0,0.04,0.08,0.12,0.16], 
                v_border_labels=['', '0.02', '0.06', '0.10','0.14', '0.18'], file_path=r'C:\Users\laurawheeler\Desktop\topoplot' + str(j) + '.png', screen_channels=True, annotate_ch_names=True)
plt.show(block = True)

In [ ]:
### Raw Trace

In [ ]:
import numpy as np
import pickle
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mne
from pylab import *

matplotlib.use("Qt5agg")

path = r'C:\Users\laurawheeler\Desktop\beta\2\raw\D1\left_r\run_02_data.npy.npz'
with open(path, 'rb') as file:
    data = np.load(file)
    data = data['arr_0']
freq = 5000

path_info = r'C:\Users\laurawheeler\Desktop\beta\2\raw\D1\left_r\run_02_hdr.pkl'
run_00_hdr = open(path_info, 'rb')
model_input = pickle.load(run_00_hdr)
ch_names = model_input['chNames']

info = mne.create_info(ch_names, 5000)
raw_data = mne.io.RawArray(data, info)
fig = mne.viz.plot_raw(raw_data)
plt.show(block = True)

In [ ]:
### Frequency Spectrum Plot

In [1]:
# Plot Frequency Spectrum of a single trial
import matplotlib.pyplot as plt

path = r'/Users/laurawheeler/Desktop/test_data/run_07_data.npy.npz' #choose individual file .npz
path_info = r'/Users/laurawheeler/Desktop/test_data/run_07_hdr.pkl' #choose individual file .pkl
fs = 5000
freq = 600

data1, data2, step4 = single_file(path, path_info, fs, freq)

freqs, psd = signal.welch(step4[0], fs=freq, nperseg=4*140)
plt.figure()
plt.semilogy(freqs, psd)

#highlight beta band
z = np.array([False] * len(freqs))
print(freqs.shape)
z[int((0.045)*len(freqs)):int((0.112)*len(freqs))] = True
plt.fill_between(freqs, psd, color='blue', alpha=0.3, where=z)

plt.xlabel('Frequency [Hz]')
plt.xlim([0, 60]) 
plt.ylim([1e-14, 1e-10]) 
plt.ylabel('Linear spectrum [dB]')
plt.show()

NameError: name 'single_file' is not defined